In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import pandas as pd
from bert_score import score
import re
import os
from pathlib import Path
import argparse
import re
import string
from collections import Counter
import pathlib
import argparse
import copy
from collections import defaultdict

In [3]:
from tqdm.auto import tqdm

In [4]:
refs=pd.read_csv("Meta-Llama-3-8B-Instruct.csv").references.tolist()

In [5]:
"""
XQuAD

Official evaluation script for v1.1 of the SQuAD dataset. """
# fork from https://huggingface.co/spaces/evaluate-metric/squad/blob/main/squad.py
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def cut_thai(text): # Add!!!
        return ' '.join(text.split())#word_tokenize(text,engine="newmm"))  # For Thai

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(cut_thai(s)))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def compute_score(references, predictions):
    f1 = exact_match = total = 0
    for _ground_truths,_prediction in list(zip(references, predictions)):
        total += 1
        ground_truths = [_ground_truths]
        prediction = _prediction
        exact_match += metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {"exact_match": exact_match, "f1": f1}

In [6]:
df=pd.read_csv("WangchanLion7B.csv")
_temp=pd.read_csv("./result/eval-gpt-4-WangchanLion7B.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='th', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("mbert-done_eval_WangchanLion7B.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.07 seconds, 93.82 sentences/sec


In [7]:
df=pd.read_csv("llama3-8b-cpt-sea-lionv2-instruct.csv")
_temp=pd.read_csv("./result/eval-gpt-4-llama3-8b-cpt-sea-lionv2-instruct.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("mbert-done_eval_llama3-8b-cpt-sea-lionv2-instruct.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.41 seconds, 241.61 sentences/sec


In [8]:
df=pd.read_csv("Meta-Llama-3.1-8B-Instruct.csv")
_temp=pd.read_csv("./result/eval-gpt-4-Meta-Llama-3.1-8B-Instruct.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("mbert-done_eval_Meta-Llama-3.1-8B-Instruct.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.59 seconds, 169.20 sentences/sec


In [9]:
df=pd.read_csv("Meta-Llama-3-8B-Instruct.csv")
_temp=pd.read_csv("./result/eval-gpt-4-Meta-Llama-3-8B-Instruct.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("mbert-done_eval_Meta-Llama-3-8B-Instruct.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.62 seconds, 161.28 sentences/sec


In [10]:
df=pd.read_csv("openthaigpt-1.0.0-beta-7b-chat-ckpt-hf.csv")
_temp=pd.read_csv("./result/eval-gpt-4-openthaigpt-1.0.0-beta-7b-chat-ckpt-hf.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("done_eval_openthaigpt-1.0.0-beta-7b-chat-ckpt-hf.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.67 seconds, 37.48 sentences/sec


In [11]:
df=pd.read_csv("SeaLLM-7B-v2.csv")
_temp=pd.read_csv("./result/eval-gpt-4-SeaLLM-7B-v2.csv")
cands=df.predictions.tolist()
df["q1"]=_temp["model_q1"]
df["q2"]=_temp["model_q2"]
df["q3"]=_temp["model_q3"]
df["q4"]=_temp["model_q4"]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
df["bertscore_p"]=[float(i)*100 for i in P]
df["bertscore_r"]=[float(i)*100 for i in R]
df["bertscore_f1"]=[float(i)*100 for i in F1]
exact_match=[]
f1=[]
for r,p in zip(df["references"],df["predictions"]):
    _temp = compute_score([r.strip()],[p.strip()])
    exact_match.append(_temp['exact_match'])
    f1.append(_temp['f1'])
df["xquad_exact_match"]=exact_match
df["xquad_f1"]=f1
df.to_excel("done_eval_SeaLLM-7B-v2.xlsx",index=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 0.57 seconds, 175.46 sentences/sec
